In [283]:
class EstadoBloques:
    def __init__(self, p=[], m=['a','b'], l=['b','c'], s=[['a','c']]):
        
        self.pinza = p
        self.mesa = m
        self.libre = l
        self.sobre = s

    def __str__(self):
        r = 'Pinza: {}\nEn mesa: {}\nLibres: {}\nSobre: {}\n'.format(self.pinza,
                [mesa for mesa in self.mesa],
                [libre for libre in self.libre],
                [sobre for sobre in self.sobre])
        
        return r
    
    def __eq__(self,other):
        return self.__str__() == other.__str__()
    
    def __hash__(self):
        return hash(self.__str__())
    
    def is_final(self):
        return self.pinza == [] and self.mesa == ['c'] and ['c','b'] in self.sobre and ['b','a'] in self.sobre and self.libre == ['a']
    
    def coger1(self,x):
        
        r = False
        
        if self.pinza == [] and x in self.libre:
            for s in self.sobre:
                if s[1] == x:
                    y,x = s
                    del self.libre[self.libre.index(x)]
                    del self.sobre[self.sobre.index(s)]
                    self.pinza.append(x)
                    self.libre.append(y)
                    r=True
                    
        return r
                    
                    
    def coger2(self,x):
        r = False
        if self.pinza == [] and x in self.libre and x in self.mesa:
            del self.libre[self.libre.index(x)]
            del self.mesa[self.mesa.index(x)]
            self.pinza.append(x)
            r = True
        return r
    
    def desapilar(self,x):
        r = False
        if x in self.pinza:
            del self.pinza[self.pinza.index(x)]
            self.mesa.append(x)
            self.libre.append(x)
            r = True
        return r
    
    def apilar(self,x,y):
        r=False
        if x in self.pinza and y in self.libre:
            del self.pinza[self.pinza.index(x)]
            del self.libre[self.libre.index(y)]
            self.libre.append(x)
            self.sobre.append([y,x])
            r=True
        return r


In [284]:
cajas = ['a','b','c']

def sucesores(estado):
    # estadoViejo = EstadoBloques(estado.pinza, estado.mesa, estado.libre, estado.sobre)
    estadoModificar = EstadoBloques(estado.pinza, estado.mesa, estado.libre, estado.sobre)
    
    pinzacopy = estado.pinza[:]
    mesacopy = estado.mesa[:]
    librecopy = estado.libre[:]
    sobrecopy = estado.sobre[:]
    
    listaSucc = []
    
    if estadoModificar.pinza:
        p = estadoModificar.pinza[0]
        for x in estadoModificar.libre:
            boo = estadoModificar.apilar(p,x)
            if boo:
                listaSucc.append(estadoModificar)
                estadoModificar = EstadoBloques(pinzacopy, mesacopy, librecopy, sobrecopy)
        estadoModificar.desapilar(p)
        listaSucc.append(estadoModificar)
        estadoModificar = EstadoBloques(pinzacopy, mesacopy, librecopy, sobrecopy)
    else:
        for x in estadoModificar.libre:
            if x in estadoModificar.mesa:
                estadoModificar.coger2(x)
                listaSucc.append(estadoModificar)
                estadoModificar = EstadoBloques(pinzacopy, mesacopy, librecopy, sobrecopy)
            else:
                estadoModificar.coger1(x)
                listaSucc.append(estadoModificar)
                estadoModificar = EstadoBloques(pinzacopy, mesacopy, librecopy, sobrecopy)
                
    return listaSucc
    

In [ ]:
def primero_profundidad():
    
    

In [285]:
prueba = EstadoBloques()
print(prueba)

Pinza: []
En mesa: ['a', 'b']
Libres: ['b', 'c']
Sobre: [['a', 'c']]



In [286]:
prueba.apilar('a','b')

False

In [287]:
prueba.coger1('c')

True

In [288]:
print(prueba.sobre)

[]


In [289]:
print(prueba)

Pinza: ['c']
En mesa: ['a', 'b']
Libres: ['b', 'a']
Sobre: []



In [290]:
prueba.desapilar('c')

True

In [291]:
print(prueba)

Pinza: []
En mesa: ['a', 'b', 'c']
Libres: ['b', 'a', 'c']
Sobre: []



In [292]:
prueba.coger2('b')

True

In [293]:
prueba.apilar('b','c')

True

In [294]:
print(prueba)

Pinza: []
En mesa: ['a', 'c']
Libres: ['a', 'b']
Sobre: [['c', 'b']]



In [295]:
prueba.is_final()

False

In [296]:
prueba.coger2('a')
prueba.apilar('a','b')

True

In [297]:
print(prueba)

Pinza: []
En mesa: ['c']
Libres: ['a']
Sobre: [['c', 'b'], ['b', 'a']]



In [298]:
prueba.is_final()

True

In [299]:
pru2 = EstadoBloques()

In [300]:
print(pru2)

Pinza: []
En mesa: ['c']
Libres: ['a']
Sobre: [['c', 'b'], ['b', 'a']]



In [301]:
lista = sucesores(pru2)

In [302]:
for j in lista:
    print(j)

Pinza: ['a']
En mesa: ['c']
Libres: ['b']
Sobre: [['c', 'b']]



In [303]:
sucesores(pru2)